# Design Pattern - Proxy

El patrón Proxy (representante o intermediario) proporciona un sustituto o intermediario para controlar el acceso a otro objeto.

En lugar de que el cliente interactúe directamente con el objeto real (llamado “sujeto real”), lo hace a través de un proxy que puede:
- Controlar el acceso 🛡️
- Añadir lógica adicional (caché, logs, validaciones…) 📝
- Retrasar la creación del objeto real ⏳
- Encapsular detalles de red 🌐

## ¿Para qué se usa?

El patrón Proxy es útil cuando quieres interceptar o controlar interacciones con un objeto sin modificarlo directamente.

Ejemplos reales comunes:
- 🛡️ Control de acceso: Comprobar permisos antes de permitir el uso de un servicio.
- 💰 Carga diferida (lazy loading): No crear un objeto pesado hasta que sea necesario.
- 📡 Proxy remoto: Representar un objeto en otro servidor o máquina.
- ⚡ Caché: Guardar resultados de operaciones costosas para no repetirlas.
- 📝 Logs y monitoreo: Registrar llamadas a un objeto real.

## Componentes clave del patrón Proxy
| Componente             | Descripción                                                               |
| ---------------------- | ------------------------------------------------------------------------- |
| **Subject (Interfaz)** | Define las operaciones que tanto el Proxy como el RealSubject implementan |
| **RealSubject**        | El objeto “real” que realiza la lógica principal                          |
| **Proxy**              | Intermediario que controla el acceso a RealSubject                        |
| **Client**             | Interactúa con el Proxy como si fuera el RealSubject                      |

## Ejemplo real en TypeScript
Imaginemos un servicio que carga datos pesados desde una API externa.
Queremos usar un Proxy para:
- No volver a llamar a la API si ya tenemos los datos (→ caché)
- Registrar cada acceso (→ logging)

### 🧠 Interfaz común (Subject)

In [1]:
interface DataSource {
  requestData(url: string): Promise<string>;
}

### 🌐 Objeto real (RealSubject)

In [2]:
class RealDataSource implements DataSource {
  async requestData(url: string): Promise<string> {
    console.log(`📡 Llamando a la API real: ${url}`);
    // Simulación de llamada HTTP costosa
    await new Promise((res) => setTimeout(res, 1000));
    return `Datos desde ${url}`;
  }
}

### 🧍‍♂️ Proxy con caché + logging

In [3]:
class DataSourceProxy implements DataSource {
  private realDataSource: RealDataSource;
  private cache: Map<string, string> = new Map();

  constructor() {
    this.realDataSource = new RealDataSource();
  }

  async requestData(url: string): Promise<string> {
    console.log(`🔍 Proxy: solicitando datos para ${url}`);

    // 1. Revisar caché
    if (this.cache.has(url)) {
      console.log(`⚡ Devolviendo datos desde caché para ${url}`);
      return this.cache.get(url)!;
    }

    // 2. Llamar al RealDataSource si no está en caché
    const data = await this.realDataSource.requestData(url);

    // 3. Guardar en caché
    this.cache.set(url, data);
    console.log(`✅ Datos guardados en caché para ${url}`);

    return data;
  }
}

### 👨‍💻 Cliente usando el Proxy

In [4]:
async function client() {
  const proxy = new DataSourceProxy();

  console.log(await proxy.requestData("https://api.com/user/1"));
  console.log(await proxy.requestData("https://api.com/user/1")); // segunda llamada → caché
  console.log(await proxy.requestData("https://api.com/user/2"));
}

client();

🔍 Proxy: solicitando datos para https://api.com/user/1
📡 Llamando a la API real: https://api.com/user/1


Promise { <pending> }

✅ Datos guardados en caché para https://api.com/user/1
Datos desde https://api.com/user/1
🔍 Proxy: solicitando datos para https://api.com/user/1
⚡ Devolviendo datos desde caché para https://api.com/user/1
Datos desde https://api.com/user/1
🔍 Proxy: solicitando datos para https://api.com/user/2
📡 Llamando a la API real: https://api.com/user/2
✅ Datos guardados en caché para https://api.com/user/2
Datos desde https://api.com/user/2


👉 Como ves, la segunda llamada al mismo recurso no toca la API real, porque el Proxy la sirvió desde caché.

### 📝 Ventajas del patrón Proxy
| Ventaja                              | Descripción                                         |
| ------------------------------------ | --------------------------------------------------- |
| 🧠 **Control centralizado**          | Intercepta todas las llamadas en un solo lugar      |
| 💾 **Caché y optimización**          | Mejora el rendimiento en operaciones repetitivas    |
| 🛡️ **Seguridad y permisos**         | Puedes controlar el acceso al objeto real           |
| 📝 **Transparencia para el cliente** | El cliente no nota si habla con Proxy o RealSubject |

In [ ]:
### ⚠️ Cuándo NO usarlo